<a href="https://colab.research.google.com/github/xchen35/Experienment_on_Relation_Extration/blob/master/TextCNN_Relational_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import re
import nltk
import subprocess

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
import datetime
import time
from sklearn.metrics import f1_score

In [0]:
class2label = {'Other': 0,
               'Message-Topic(e1,e2)': 1, 'Message-Topic(e2,e1)': 2,
               'Product-Producer(e1,e2)': 3, 'Product-Producer(e2,e1)': 4,
               'Instrument-Agency(e1,e2)': 5, 'Instrument-Agency(e2,e1)': 6,
               'Entity-Destination(e1,e2)': 7, 'Entity-Destination(e2,e1)': 8,
               'Cause-Effect(e1,e2)': 9, 'Cause-Effect(e2,e1)': 10,
               'Component-Whole(e1,e2)': 11, 'Component-Whole(e2,e1)': 12,
               'Entity-Origin(e1,e2)': 13, 'Entity-Origin(e2,e1)': 14,
               'Member-Collection(e1,e2)': 15, 'Member-Collection(e2,e1)': 16,
               'Content-Container(e1,e2)': 17, 'Content-Container(e2,e1)': 18}

label2class = {0: 'Other',
               1: 'Message-Topic(e1,e2)', 2: 'Message-Topic(e2,e1)',
               3: 'Product-Producer(e1,e2)', 4: 'Product-Producer(e2,e1)',
               5: 'Instrument-Agency(e1,e2)', 6: 'Instrument-Agency(e2,e1)',
               7: 'Entity-Desatination(e1,e2)', 8: 'Entity-Destination(e2,e1)',
               9: 'Cause-Effect(e1,e2)', 10: 'Cause-Effect(e2,e1)',
               11: 'Component-Whole(e1,e2)', 12: 'Component-Whole(e2,e1)',
               13: 'Entity-Origin(e1,e2)', 14: 'Entity-Origin(e2,e1)',
               15: 'Member-Collection(e1,e2)', 16: 'Member-Collection(e2,e1)',
               17: 'Content-Container(e1,e2)', 18: 'Content-Container(e2,e1)'}

In [0]:
def load_word2vec(embedding_path, embedding_dim, vocab):
    # initial matrix with random uniform
    initW = np.random.randn(len(vocab.vocabulary_), embedding_dim).astype(np.float32) / np.sqrt(len(vocab.vocabulary_))
    # load any vectors from the word2vec
    print("Load word2vec file {0}".format(embedding_path))
    with open(embedding_path, "rb") as f:
        header = f.readline()
        vocab_size, layer_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer_size
        for line in range(vocab_size):
            word = []
            while True:
                ch = f.read(1).decode('latin-1')
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)
            idx = vocab.vocabulary_.get(word)
            if idx != 0:
                initW[idx] = np.fromstring(f.read(binary_len), dtype='float32')
            else:
                f.read(binary_len)
    return initW

In [0]:
def clean_str(text):
    text = text.lower()
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"that's", "that is ", text)
    text = re.sub(r"there's", "there is ", text)
    text = re.sub(r"it's", "it is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    return text.strip()



In [0]:
def load_data_and_labels(path):
    data = []
    lines = [line.strip() for line in open(path)]
    max_sentence_length = 0
    for idx in range(0, len(lines), 4):
        id = lines[idx].split("\t")[0]
        relation = lines[idx + 1]

        sentence = lines[idx].split("\t")[1][1:-1]
        sentence = sentence.replace('<e1>', ' _e11_ ')
        sentence = sentence.replace('</e1>', ' _e12_ ')
        sentence = sentence.replace('<e2>', ' _e21_ ')
        sentence = sentence.replace('</e2>', ' _e22_ ')

        sentence = clean_str(sentence)
        tokens = nltk.word_tokenize(sentence)
        if max_sentence_length < len(tokens):
            max_sentence_length = len(tokens)
        e1 = tokens.index("e12") - 1
        e2 = tokens.index("e22") - 1
        sentence = " ".join(tokens)

        data.append([id, sentence, e1, e2, relation])

    print(path)
    print("max sentence length = {}\n".format(max_sentence_length))

    df = pd.DataFrame(data=data, columns=["id", "sentence", "e1", "e2", "relation"])

    pos1, pos2 = get_relative_position(df, 90)

    df['label'] = [class2label[r] for r in df['relation']]

    # Text Data
    x_text = df['sentence'].tolist()

    # Label Data
    y = df['label']
    labels_flat = y.values.ravel()
    labels_count = np.unique(labels_flat).shape[0]

    # convert class labels from scalars to one-hot vectors
    # 0  => [1 0 0 0 0 ... 0 0 0 0 0]
    # 1  => [0 1 0 0 0 ... 0 0 0 0 0]
    # ...
    # 18 => [0 0 0 0 0 ... 0 0 0 0 1]
    def dense_to_one_hot(labels_dense, num_classes):
        num_labels = labels_dense.shape[0]
        index_offset = np.arange(num_labels) * num_classes
        labels_one_hot = np.zeros((num_labels, num_classes))
        labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
        return labels_one_hot

    labels = dense_to_one_hot(labels_flat, labels_count)
    labels = labels.astype(np.uint8)

    return x_text, labels, pos1, pos2

In [0]:
def get_relative_position(df, max_sentence_length):
    # Position data
    pos1 = []
    pos2 = []
    for df_idx in range(len(df)):
        sentence = df.iloc[df_idx]['sentence']
        tokens = nltk.word_tokenize(sentence)
        e1 = df.iloc[df_idx]['e1']
        e2 = df.iloc[df_idx]['e2']

        p1 = ""
        p2 = ""
        for word_idx in range(len(tokens)):
            p1 += str((max_sentence_length - 1) + word_idx - e1) + " "
            p2 += str((max_sentence_length - 1) + word_idx - e2) + " "
        pos1.append(p1)
        pos2.append(p2)

    return pos1, pos2

In [0]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [0]:
trainFile = '/content/drive/My Drive/SemEval2010_task8_all_data/SemEval2010_task8_training/TRAIN_FILE.TXT'
testFile = '/content/drive/My Drive/SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT'

x_text, y, pos1, pos2 = load_data_and_labels(testFile)

/content/drive/My Drive/SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT
max sentence length = 68



**Develop TextCNN model**

In [0]:
class TextCNN:
    def __init__(self, sequence_length, num_classes,
                 text_vocab_size, text_embedding_size, pos_vocab_size, pos_embedding_size,
                 filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_text = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_text')
        self.input_p1 = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_p1')
        self.input_p2 = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_p2')
        self.input_y = tf.placeholder(tf.float32, shape=[None, num_classes], name='input_y')
        self.dropout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')

        initializer = tf.keras.initializers.glorot_normal

        # Embedding layer
        with tf.variable_scope("text-embedding"):
            self.W_text = tf.Variable(tf.random_uniform([text_vocab_size, text_embedding_size], -0.25, 0.25), name="W_text")
            self.text_embedded_chars = tf.nn.embedding_lookup(self.W_text, self.input_text)
            self.text_embedded_chars_expanded = tf.expand_dims(self.text_embedded_chars, -1)

        with tf.variable_scope("position-embedding"):
            self.W_pos = tf.get_variable("W_pos", [pos_vocab_size, pos_embedding_size], initializer=initializer())
            self.p1_embedded_chars = tf.nn.embedding_lookup(self.W_pos, self.input_p1)
            self.p2_embedded_chars = tf.nn.embedding_lookup(self.W_pos, self.input_p2)
            self.p1_embedded_chars_expanded = tf.expand_dims(self.p1_embedded_chars, -1)
            self.p2_embedded_chars_expanded = tf.expand_dims(self.p2_embedded_chars, -1)

        self.embedded_chars_expanded = tf.concat([self.text_embedded_chars_expanded,
                                                  self.p1_embedded_chars_expanded,
                                                  self.p2_embedded_chars_expanded], 2)
        _embedding_size = text_embedding_size + 2*pos_embedding_size

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.variable_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                conv = tf.layers.conv2d(self.embedded_chars_expanded, num_filters, [filter_size, _embedding_size],
                                        kernel_initializer=initializer(), activation=tf.nn.relu, name="conv")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(conv, ksize=[1, sequence_length - filter_size + 1, 1, 1],
                                        strides=[1, 1, 1, 1], padding='VALID', name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.variable_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final scores and predictions
        with tf.variable_scope("output"):
            self.logits = tf.layers.dense(self.h_drop, num_classes, kernel_initializer=initializer())
            self.predictions = tf.argmax(self.logits, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.variable_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, labels=self.input_y)
            self.l2 = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()])
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * self.l2

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name="accuracy")

In [0]:
 # Build vocabulary
text_vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(90)
x = np.array(list(text_vocab_processor.fit_transform(x_text)))
print("Text Vocabulary Size: {:d}".format(len(text_vocab_processor.vocabulary_)))
print("x = {0}".format(x.shape))
print("y = {0}".format(y.shape))
print("")

Text Vocabulary Size: 10470
x = (2717, 90)
y = (2717, 19)



In [0]:
 # Build vocabulary
pos_vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(90)
pos_vocab_processor.fit(pos1 + pos2)
p1 = np.array(list(pos_vocab_processor.transform(pos1)))
p2 = np.array(list(pos_vocab_processor.transform(pos2)))
print("Position Vocabulary Size: {:d}".format(len(pos_vocab_processor.vocabulary_)))
print("position_1 = {0}".format(p1.shape))
print("position_2 = {0}".format(p2.shape))
print("")

Position Vocabulary Size: 121
position_1 = (2717, 90)
position_2 = (2717, 90)



In [0]:
# Randomly shuffle data to split into train and test(dev)
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
p1_shuffled = p1[shuffle_indices]
p2_shuffled = p2[shuffle_indices]
y_shuffled = y[shuffle_indices]

In [0]:
# Split train/test set
dev_sample_index = -1 * int(0.1 * float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
p1_train, p1_dev = p1_shuffled[:dev_sample_index], p1_shuffled[dev_sample_index:]
p2_train, p2_dev = p2_shuffled[:dev_sample_index], p2_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]
print("Train/Dev split: {:d}/{:d}\n".format(len(y_train), len(y_dev)))

Train/Dev split: 2446/271



**Build Model**

In [0]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
        allow_soft_placement=True,
        log_device_placement=False)
    session_conf.gpu_options.allow_growth = True
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            text_vocab_size=len(text_vocab_processor.vocabulary_),
            text_embedding_size=300,
            pos_vocab_size=len(pos_vocab_processor.vocabulary_),
            pos_embedding_size=50,
            filter_sizes=list(map(int, "2,3,4,5".split(","))),
            num_filters=128,
            l2_reg_lambda=1e-5)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdadeltaOptimizer(1.0, 0.9, 1e-6)
        gvs = optimizer.compute_gradients(cnn.loss)
        capped_gvs = [(tf.clip_by_value(grad, -1.0, 1.0), var) for grad, var in gvs]
        train_op = optimizer.apply_gradients(capped_gvs, global_step=global_step)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join("drive/My Drive", "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)

        # Write vocabulary
        text_vocab_processor.save(os.path.join(out_dir, "text_vocab"))
        pos_vocab_processor.save(os.path.join(out_dir, "pos_vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        # Pre-trained word2vec
        if None:
            pretrain_W = utils.load_word2vec(None, 300, text_vocab_processor)
            sess.run(cnn.W_text.assign(pretrain_W))
            print("Success to load pre-trained word2vec model!\n")

        # Generate batches
        batches = batch_iter(list(zip(x_train, p1_train, p2_train, y_train)),
                                          20, 100)
        # Training loop. For each batch...
        best_f1 = 0.0  # For save checkpoint(model)
        for batch in batches:
            x_batch, p1_batch, p2_batch, y_batch = zip(*batch)
            # Train
            feed_dict = {
                cnn.input_text: x_batch,
                cnn.input_p1: p1_batch,
                cnn.input_p2: p2_batch,
                cnn.input_y: y_batch,
                cnn.dropout_keep_prob: 0.5
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy], feed_dict)
            train_summary_writer.add_summary(summaries, step)

            # Training log display
            if step % 10 == 0:
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

            # Evaluation
            if step % 100 == 0:
                print("\nEvaluation:")
                feed_dict = {
                    cnn.input_text: x_dev,
                    cnn.input_p1: p1_dev,
                    cnn.input_p2: p2_dev,
                    cnn.input_y: y_dev,
                    cnn.dropout_keep_prob: 1.0
                }
                summaries, loss, accuracy, predictions = sess.run(
                    [dev_summary_op, cnn.loss, cnn.accuracy, cnn.predictions], feed_dict)
                dev_summary_writer.add_summary(summaries, step)

                time_str = datetime.datetime.now().isoformat()
                f1 = f1_score(np.argmax(y_dev, axis=1), predictions, labels=np.array(range(1, 19)), average="macro")
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                print("[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): {:g}\n".format(f1))

                # Model checkpoint
                if best_f1 < f1:
                    best_f1 = f1
                    path = saver.save(sess, checkpoint_prefix + "-{:.3g}".format(best_f1), global_step=step)
                    print("Saved model checkpoint to {}\n".format(path))

Writing to /content/drive/My Drive/runs/1554407997

2019-04-04T19:59:58.513649: step 10, loss 3.03723, acc 0.25
2019-04-04T19:59:58.736299: step 20, loss 2.506, acc 0.4
2019-04-04T19:59:58.934491: step 30, loss 1.90786, acc 0.6
2019-04-04T19:59:59.125092: step 40, loss 2.3436, acc 0.45
2019-04-04T19:59:59.309128: step 50, loss 2.18892, acc 0.4
2019-04-04T19:59:59.498203: step 60, loss 3.07166, acc 0.25
2019-04-04T19:59:59.683537: step 70, loss 2.12132, acc 0.3
2019-04-04T19:59:59.872661: step 80, loss 2.45624, acc 0.25
2019-04-04T20:00:00.062702: step 90, loss 2.08163, acc 0.45
2019-04-04T20:00:00.244896: step 100, loss 2.17, acc 0.45

Evaluation:
2019-04-04T20:00:00.325883: step 100, loss 2.00576, acc 0.527675
[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): 0.404361



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Saved model checkpoint to /content/drive/My Drive/runs/1554407997/checkpoints/model-0.404-100

2019-04-04T20:00:00.766424: step 110, loss 2.10452, acc 0.45
2019-04-04T20:00:00.950391: step 120, loss 1.44472, acc 0.6
2019-04-04T20:00:01.130336: step 130, loss 1.68129, acc 0.6
2019-04-04T20:00:01.318766: step 140, loss 1.91809, acc 0.5
2019-04-04T20:00:01.503113: step 150, loss 1.74221, acc 0.55
2019-04-04T20:00:01.692868: step 160, loss 1.71685, acc 0.65
2019-04-04T20:00:01.883766: step 170, loss 1.99708, acc 0.25
2019-04-04T20:00:02.071765: step 180, loss 1.63917, acc 0.55
2019-04-04T20:00:02.258363: step 190, loss 1.67851, acc 0.55
2019-04-04T20:00:02.448223: step 200, loss 1.41328, acc 0.6

Evaluation:
2019-04-04T20:00:02.479971: step 200, loss 1.7244, acc 0.560886
[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): 0.438528

Saved model checkpoint to /content/drive/My Drive/runs/1554407997/checkpoints/model-0.439-200

2019-04-04T20:00:02.943339: step 210, loss 1.20927

**Eval()**

In [0]:
# Map data into vocabulary
text_path = "drive/My Drive/runs/1554407997/text_vocab"
text_vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor.restore(text_path)
x = np.array(list(text_vocab_processor.transform(x_text)))

In [0]:
position_path = "drive/My Drive/runs/1554407997/pos_vocab"
position_vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor.restore(position_path)
p1 = np.array(list(position_vocab_processor.transform(pos1)))
p2 = np.array(list(position_vocab_processor.transform(pos2)))

In [0]:
checkpoint_file = tf.train.latest_checkpoint("drive/My Drive/runs/1554407997/checkpoints")

In [0]:
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
        allow_soft_placement=True,
        log_device_placement=False)
    session_conf.gpu_options.allow_growth = True
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_text = graph.get_operation_by_name("input_text").outputs[0]
        input_p1 = graph.get_operation_by_name("input_p1").outputs[0]
        input_p2 = graph.get_operation_by_name("input_p2").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        # Generate batches for one epoch
        batches = batch_iter(list(zip(x, p1, p2)), 20, 1, shuffle=False)

        # Collect the predictions here
        preds = []
        for batch in batches:
            x_batch, p1_batch, p2_batch = zip(*batch)
            pred = sess.run(predictions, {input_text: x_batch,
                                          input_p1: p1_batch,
                                          input_p2: p2_batch,
                                          dropout_keep_prob: 1.0})
            preds.append(pred)
        preds = np.concatenate(preds)
        truths = np.argmax(y, axis=1)

        prediction_path = os.path.join("drive/My Drive/runs/1554407997", "predictions.txt")
        truth_path = os.path.join("drive/My Drive/runs/1554407997", "ground_truths.txt")
        prediction_file = open(prediction_path, 'w')
        truth_file = open(truth_path, 'w')
        for i in range(len(preds)):
            prediction_file.write("{}\t{}\n".format(i, label2class[preds[i]]))
            truth_file.write("{}\t{}\n".format(i, label2class[truths[i]]))
        prediction_file.close()
        truth_file.close()

        perl_path = os.path.join(os.path.curdir,
                                 "SemEval2010_task8_all_data",
                                 "SemEval2010_task8_scorer-v1.2",
                                 "semeval2010_task8_scorer-v1.2.pl")
        process = subprocess.Popen(["perl", perl_path, prediction_path, truth_path], stdout=subprocess.PIPE)
        for line in str(process.communicate()[0].decode("utf-8")).split("\\n"):
            print(line)

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/runs/1554407997/checkpoints/model-0.627-7000



In [0]:
!ls "drive/My Drive/runs/1554407997/checkpoints"

checkpoint			      model-0.61-3000.index
model-0.591-2200.data-00000-of-00001  model-0.61-3000.meta
model-0.591-2200.index		      model-0.616-4900.data-00000-of-00001
model-0.591-2200.meta		      model-0.616-4900.index
model-0.594-2400.data-00000-of-00001  model-0.616-4900.meta
model-0.594-2400.index		      model-0.627-7000.data-00000-of-00001
model-0.594-2400.meta		      model-0.627-7000.index
model-0.61-3000.data-00000-of-00001   model-0.627-7000.meta
